In [1]:
from tensorflow.keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
import glob
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [14]:
IMG_SIZE = 256
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

In [2]:
image_directory = './data/images'
mask_directory = './data/masks'

In [3]:
image_names = glob.glob("./data/images/*.png")
mask_names = glob.glob("./data/masks/*.png")

In [4]:
image_names.sort()
mask_names.sort()
subset_length = 100
image_names = image_names[:subset_length]
mask_names = mask_names[:subset_length]

In [5]:
def display_image(cv2_img,cv2_mask):
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    plt.imshow(cv2_img, cmap='gray')
    plt.subplot(122)
    plt.imshow(cv2_mask, cmap='gray')
    plt.show()

In [6]:
# reading images and mask
images = [cv2.imread(img, 0) for img in image_names]
masks = [cv2.imread(img,0) for img in mask_names]

In [7]:
# converting images into a numpy array and expanding the dimension of each image
image_dataset = np.array(images)
image_dataset = np.expand_dims(image_dataset, axis = 3)
mask_dataset = np.array(masks)
mask_dataset = np.expand_dims(mask_dataset,axis=3)

Using tf.image for preprocessing images

In [11]:
def resize_and_rescale(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  image = (image / 255.0)
  return image

In [12]:
def augment(image_label, seed):
  
  image, label = image_label
  
  image = resize_and_rescale(image)
  label = resize_and_rescale(label)

  image = tf.image.stateless_random_crop(value=image,size=(IMG_SIZE,IMG_SIZE,1),seed=seed)
  label = tf.image.stateless_random_crop(value=label,size=(IMG_SIZE,IMG_SIZE,1),seed=seed)

  image = tf.image.stateless_random_flip_left_right(image,seed=seed)
  label = tf.image.stateless_random_flip_left_right(label,seed=seed)

  image = tf.image.stateless_random_flip_up_down(image,seed=seed)
  label = tf.image.stateless_random_flip_up_down(label,seed=seed)
  
# [-max_delta, max_delta]
  image = tf.image.stateless_random_brightness(image,max_delta=0.2,seed=seed)
  label = tf.image.stateless_random_brightness(label,max_delta=0.2,seed=seed)

# [1.0 - lower, 1.0 + upper]
  image = tf.image.stateless_random_contrast(image,lower=0.2,upper=0.5,seed=seed)
  label = tf.image.stateless_random_contrast(label,lower=0.2,upper=0.5,seed=seed)

  return image, label

In [19]:
random_number_generator = tf.random.Generator.from_seed(123, alg='philox')
def transform_images(x, y):
  seed = random_number_generator.make_seeds(2)[0]
  image, label = augment((x, y), seed)
  return image, label

In [15]:
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.20, random_state = 42)

In [16]:
# converting images from numpy to tf.Data
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [20]:
# Applying augmentation, shuffling and batching

train_ds = (
    train_dataset
    .shuffle(50)
    .repeat(20)
    .map(transform_images, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

test_ds = (
    test_dataset
    .shuffle(50)
    .repeat(20)
    .map(transform_images, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)